In [45]:
import mysql.connector

Connecting with database

In [46]:
import db_credentials as creds
mydb = mysql.connector.connect(
    host=creds.host,
    user=creds.user,
    password=creds.password,
    database=creds.database,
    port = creds.port
)

In [47]:
mycursor = mydb.cursor()

In [48]:
# testing
mycursor.execute("SHOW TABLES")

In [49]:
for x in mycursor:
    print(x)

('ayah_edition',)
('ayahs',)
('unique_eng_words',)


## Term document matrix

In [50]:
# create np array of size 6236x5480
import numpy as np
term_matrix = np.zeros((6236, 5480))
term_matrix.shape

(6236, 5480)

In [51]:
mycursor.execute("SELECT * FROM ayah_edition")

In [52]:
# converting to numpy array
ayahs_records = np.array(mycursor.fetchall())
ayahs_records[0][3]

'In the name of Allah, most benevolent, ever-merciful.'

## Getting just ayah strings from records

In [53]:
ayahs = ayahs_records[:,3]
ayahs

array(['In the name of Allah, most benevolent, ever-merciful.',
       'ALL PRAISE BE to Allah, Lord of all the worlds,',
       'Most beneficent, ever-merciful,', ...,
       'From the evil of him who breathes temptations into the minds of men,',
       'Who suggests evil thoughts to the hearts of men --',
       'From among the jinns and men.'], dtype=object)

## Removing symbols from ayah strings

In [54]:
# in each record using regex extract only words without comma, dot, quotations marks and apostrophe and lower case them using numpy
import re
import numpy as np
import string

def clean_text(text):
    text = text.lower()
    text = re.sub(r'-', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text

# apply the function to all the strings inside numpy array
vfunc = np.vectorize(clean_text)
cleaned_ayahs = vfunc(ayahs)
cleaned_ayahs[0]

'in the name of allah most benevolent ever merciful'

## Getting separate words

In [55]:
words_array = np.char.split(cleaned_ayahs.astype(str))
words_array

array([list(['in', 'the', 'name', 'of', 'allah', 'most', 'benevolent', 'ever', 'merciful']),
       list(['all', 'praise', 'be', 'to', 'allah', 'lord', 'of', 'all', 'the', 'worlds']),
       list(['most', 'beneficent', 'ever', 'merciful']), ...,
       list(['from', 'the', 'evil', 'of', 'him', 'who', 'breathes', 'temptations', 'into', 'the', 'minds', 'of', 'men']),
       list(['who', 'suggests', 'evil', 'thoughts', 'to', 'the', 'hearts', 'of', 'men']),
       list(['from', 'among', 'the', 'jinns', 'and', 'men'])],
      dtype=object)

## Getting unique words from the database

In [61]:
mycursor.execute("SELECT * FROM unique_eng_words")

In [62]:
unique_words_arr = np.array(mycursor.fetchall())
unique_words_arr

array([['1', 'a'],
       ['2', 'aaron'],
       ['3', 'abandon'],
       ...,
       ['5478', 'zakat'],
       ['5479', 'zaqqum'],
       ['5480', 'zodiac']], dtype='<U15')

# Filling term document matrix
Remember that inside the database, the indexing is starting from 1 but in the code, it is 0. Lets say we want to see the 'the' word inside the unique words inside the database. Its id is 4874 but here in the code, its index is 4873

In [67]:
for idx, ayah in enumerate(words_array):
    for word in ayah:
        if word in unique_words_arr[:,1]:
            term_matrix[idx,np.where(unique_words_arr[:,1] == word)[0]] = 1
        
term_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [72]:
term_matrix[0][4873]    # finding 'the' in the first ayah

1.0